### Imports

In [51]:
import pandas as pd
from pandas_datareader import data
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from scipy import sparse
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
import time
from sklearn.metrics import mean_squared_error

### Obtain and Prepare Data

In [52]:
start = datetime.strptime('2020-01-02','%Y-%m-%d')
end = datetime.strptime('2022-02-28','%Y-%m-%d')

df = data.DataReader('EURUSD=X',start=start, end=end, data_source='yahoo')
df = df[:1500]['Close']

In [53]:
scaler=MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(np.array(df).reshape(-1,1))

In [54]:
training_size=int(len(data)*0.65)
test_size=len(data)-training_size
train_data,test_data=data[0:training_size,:],data[training_size:len(data),:]

In [55]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i+time_step,0])
    return np.array(dataX), np.array(dataY)

### Reservoir Implementation

In [56]:
class Reservoir(object):
    """
    Build a reservoir and evaluate internal states
    
    Parameters:
        n_internal_units = processing units in the reservoir
        spectral_radius = largest eigenvalue of the reservoir matrix of connection weights
        leak = amount of leakage in the reservoir state update (optional)
        connectivity = percentage of nonzero connection weights (unused in circle reservoir)
        input_scaling = scaling of the input connection weights
        noise_level = deviation of the Gaussian noise injected in the state update
        circle = generate determinisitc reservoir with circle topology
    """
    
    def __init__(self, n_internal_units=100, spectral_radius=0.99, leak=None,
                 connectivity=0.3, input_scaling=0.2, noise_level=0.01, circle=False):
        
        # Initialize attributes
        self._n_internal_units = n_internal_units
        self._input_scaling = input_scaling
        self._noise_level = noise_level
        self._leak = leak

        # Input weights depend on input size: they are set when data is provided
        self._input_weights = None

        # Generate internal weights
        if circle:
            self._internal_weights = self._initialize_internal_weights_Circ(
                    n_internal_units,
                    spectral_radius)
        else:
            self._internal_weights = self._initialize_internal_weights(
                n_internal_units,
                connectivity,
                spectral_radius)


    def _initialize_internal_weights_Circ(self, n_internal_units, spectral_radius):
        
        internal_weights = np.zeros((n_internal_units, n_internal_units))
        internal_weights[0,-1] = spectral_radius
        for i in range(n_internal_units-1):
            internal_weights[i+1,i] = spectral_radius
                
        return internal_weights
    
    
    def _initialize_internal_weights(self, n_internal_units,
                                     connectivity, spectral_radius):

        # Generate sparse, uniformly distributed weights.
        internal_weights = sparse.rand(n_internal_units,
                                       n_internal_units,
                                       density=connectivity).todense()

        # Ensure that the nonzero values are uniformly distributed in [-0.5, 0.5]
        internal_weights[np.where(internal_weights > 0)] -= 0.5
        
        # Adjust the spectral radius.
        E, _ = np.linalg.eig(internal_weights)
        e_max = np.max(np.abs(E))
        internal_weights /= np.abs(e_max)/spectral_radius       

        return internal_weights


    def _compute_state_matrix(self, X, n_drop=0):
        N, T, _ = X.shape
        previous_state = np.zeros((N, self._n_internal_units), dtype=float)

        # Storage
        state_matrix = np.empty((N, T - n_drop, self._n_internal_units), dtype=float)
        for t in range(T):
            current_input = X[:, t, :]

            # Calculate state
            state_before_tanh = self._internal_weights.dot(previous_state.T) + self._input_weights.dot(current_input.T)

            # Add noise
            state_before_tanh += np.random.rand(self._n_internal_units, N)*self._noise_level

            # Apply nonlinearity and leakage (optional)
            if self._leak is None:
                previous_state = np.tanh(state_before_tanh).T
            else:
                previous_state = (1.0 - self._leak)*previous_state + np.tanh(state_before_tanh).T

            # Store everything after the dropout period
            if (t > n_drop - 1):
                state_matrix[:, t - n_drop, :] = previous_state

        return state_matrix


    def get_states(self, X, n_drop=0, bidir=True):
        N, T, V = X.shape
        if self._input_weights is None:
            self._input_weights = (2.0*np.random.binomial(1, 0.5 , [self._n_internal_units, V]) - 1.0)*self._input_scaling

        # compute sequence of reservoir states
        states = self._compute_state_matrix(X, n_drop)
    
        # reservoir states on time reversed input
        if bidir is True:
            X_r = X[:, ::-1, :]
            states_r = self._compute_state_matrix(X_r, n_drop)
            states = np.concatenate((states, states_r), axis=2)

        return states
    
    def getReservoirEmbedding(self, X,pca, ridge_embedding,  n_drop=5, bidir=True, test = False):

        res_states = self.get_states(X, n_drop=5, bidir=True)


        N_samples = res_states.shape[0]
        res_states = res_states.reshape(-1, res_states.shape[2])                   
        # ..transform..
        if test:
            red_states = pca.transform(res_states)
        else:
            red_states = pca.fit_transform(res_states)          
        # ..and put back in tensor form
        red_states = red_states.reshape(N_samples,-1,red_states.shape[1])  

        coeff_tr = []
        biases_tr = []   

        for i in range(X.shape[0]):
            ridge_embedding.fit(red_states[i, 0:-1, :], red_states[i, 1:, :])
            coeff_tr.append(ridge_embedding.coef_.ravel())
            biases_tr.append(ridge_embedding.intercept_.ravel())
        print(np.array(coeff_tr).shape,np.array(biases_tr).shape)
        input_repr = np.concatenate((np.vstack(coeff_tr), np.vstack(biases_tr)), axis=1)
        return input_repr

In [57]:
time_step = 100
mse_vals=[]
pca_param=[10,20,30,40,50,60,70,80,90,100]
alpha_ridges = [10,20,30,40,50]
internals=[10,20,40,50,100,200,300,500]
leaks=[0.1,0.2]
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1],1)
for pca_val in pca_param:
    for alpha_ridge in alpha_ridges:
        for internal in internals:
            for leaking in leaks:
                if pca_val>=internal:
                    continue
                pca = PCA(n_components=pca_val)                      
                ridge_embedding = Ridge(alpha=alpha_ridge, fit_intercept=True)
                readout = Ridge(alpha=alpha_ridge)
                res = Reservoir(n_internal_units=internal, spectral_radius=0.2, leak=leaking,
                        connectivity=0.85, input_scaling=0.1, noise_level=0.02, circle=False)
                input_repr = res.getReservoirEmbedding(X_train,pca, ridge_embedding,  n_drop=20, bidir=True, test = False)
                input_repr_te = res.getReservoirEmbedding(X_test,pca, ridge_embedding,  n_drop=20, bidir=True, test = True)
                readout.fit(input_repr,y_train)
                pred = readout.predict(input_repr_te)
                test_predict=readout.predict(input_repr_te)
                mse_iter=mean_squared_error(y_test, test_predict)
                params=[pca_val,alpha_ridge,internal,leaking]
                mse_vals.append([mse_iter,params])

(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)
(265, 100) (265, 10)
(97, 100) (97, 10)


(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (265, 40)
(97, 1600) (97, 40)
(265, 1600) (2

(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (97, 90)
(265, 8100) (265, 90)
(97, 8100) (

In [58]:
sorted(mse_vals)

[[0.014860707926168318, [40, 30, 200, 0.1]],
 [0.01656266126667776, [20, 20, 500, 0.1]],
 [0.01873641501278046, [10, 40, 500, 0.1]],
 [0.018745220698670628, [10, 50, 500, 0.1]],
 [0.01927336354735676, [70, 20, 200, 0.1]],
 [0.023371296278626663, [90, 50, 100, 0.1]],
 [0.02340586057159739, [40, 10, 300, 0.2]],
 [0.024053545977175184, [20, 30, 300, 0.1]],
 [0.024357453519988196, [80, 10, 300, 0.2]],
 [0.02441931066471336, [100, 10, 300, 0.2]],
 [0.024898708693570727, [10, 40, 100, 0.1]],
 [0.024922789787923588, [50, 20, 500, 0.1]],
 [0.024928687757774248, [30, 10, 50, 0.1]],
 [0.02593959372694021, [60, 20, 300, 0.1]],
 [0.026394893247051857, [50, 10, 100, 0.1]],
 [0.02697225008012701, [90, 50, 500, 0.1]],
 [0.026989770509124867, [70, 10, 300, 0.2]],
 [0.027166104162312327, [40, 10, 500, 0.2]],
 [0.027778987921709426, [70, 10, 200, 0.2]],
 [0.027917332574016004, [10, 20, 500, 0.1]],
 [0.028105230521197212, [20, 50, 200, 0.1]],
 [0.028133511260767845, [60, 20, 500, 0.1]],
 [0.0281373501762